<a href="https://colab.research.google.com/github/IIF0403/Thesis/blob/main/SimSiam_u16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Check GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [24]:
import numpy as np
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as F
from google.colab import files
from google.colab import output
from google.colab import drive
from torch.nn.utils.rnn import pack_sequence
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from sklearn.model_selection import train_test_split
from copy import deepcopy
import random
from datetime import datetime
from sklearn.metrics import accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [42]:
##### DATA #######

### Function to load a UCR time_series dataset from my github ###
def load_dataset(Dataset, train):
  #Dataset: The name of the dataset to load. Example: "ECG5000"

  #github URL
  url_raw = 'https://raw.githubusercontent.com/IIF0403/Thesis/main/data/'
  url_train = url_raw + Dataset+'/'+Dataset+'_TRAIN'
  url_test = url_raw + Dataset+'/'+Dataset+'_TEST'

  #Loading the data
  #data_train = pd.read_csv(url_train,header=None)
  #data_test = pd.read_csv(url_test, header=None)
  #data = pd.concat((data_train, data_test))

  if (train ==True): #bruker test_settene til train fordi de er større
    data = pd.read_csv(url_test, header=None)
  else:
    data = pd.read_csv(url_train, header=None)

  #Want all datasets to have classes as integers starting from 0 
  Y = data.values[:,0]
  classes = len(np.unique(Y))
  Y_transformed = ( (Y-Y.min())/(Y.max()-Y.min()) )*(classes-1)
  data[data.columns[0]] = Y_transformed

  #Inserting the name of the dataset as a column (for later use, when several datasets will be combined)
  data.insert(loc=0, column = "Dataset", value=Dataset) 

  #Inserting the length of the time series T as a column (for later use)
  T = data.shape[1]-2 #The length of the time series
  data.insert(loc=1, column = "T", value = T) 

  return data, classes


### Function to make a big dataset out of several datasets loaded from github ###
def make_big_dataset(Datasets, train):
  #Datasets: A list of the name of the datasets to load. Example: ["ECG5000", "FordA", "FordB"]

  #Loading first dataset
  data, classes =  load_dataset(Datasets[0], train)
  used_classes = classes #keeping track of the class-labels already used

  #Loading each of the datasets in the list and combining them all togehter in a big dataset
  for i in range(1, len(Datasets)):
    #loading the i´th dataset
    Dataset = Datasets[i]
    dataset, classes= load_dataset(Dataset, train)

    #Need to change the class-labels such that all the class-labels of the different datasets differ from eachother
    labels = dataset.values[:,2]
    transformed_labels = labels + used_classes 
    dataset[dataset.columns[2]] = transformed_labels
    
    used_classes += classes #keeping track of the class-labels already used

    data = pd.concat((data, dataset)) #adding the new dataset to the big dataset
  
  return data, used_classes



### Dataset class ###
class Timeseries_Dataset(Dataset):
  def __init__(self, Datasets, train = True, Drive=False, Save=None, transform=None):
    #Datasets: a list with the name of the datasets, can be a list og several or one dataset. If it is several datasets, they will be made into one big dataset
    #Drive: True means loading the already saved dataset from google drive, false means loading data from github
    #Save: name of new dataset, if we want to save the new dataset to google drive
    #transform: a given transformation of the data

    ##Loading the data
    #if the data is saved in Google Drive, load the data from Drive
    if (Drive == True):
      Dataset = Datasets[0]
      print("Loading '",Dataset,"' from Google Drive")
      #drive.mount("/content/gdrive")
      #data = pd.read_csv('/content/gdrive/My Drive/Datasets/'+Dataset+'.csv')

      drive.mount('/content/drive')
      data = pd.read_csv('/content/drive/MyDrive/Datasets/'+Dataset+'.csv')

      classes = len(np.unique(data.values[:,2]))
    
    #else load data from github
    else:
      #If "Datasets" contains several datasets then make a big dataset 
      if (len(Datasets)>1):
        print("Loading and combining ", Datasets," from github")
        data, classes = make_big_dataset(Datasets, train)
      
      #If "Datasets" only contains one dataset, load the dataset from github
      else:
        Dataset = Datasets[0]
        print("Loading '",Dataset,"' from github")
        data, classes = load_dataset(Dataset, train)
    
    #Save new dataset to Google drive as 'name' if Save = 'name' and not None
    if (Save!=None): #Save the new dataset to google drive as Save
      print("Saving new dataset to google drive")
      #drive.mount("/content/gdrive")
      #data.to_csv('/content/gdrive/My Drive/Datasets/'+Save+'.csv', index=False)

      drive.mount('/content/drive')
      data.to_csv('/content/drive/MyDrive/Datasets/'+Save+'.csv', index=False)
    
    self.dataframe = data
    self.transform = transform
    self.classes = classes
    self.Datasets = Datasets

    
  #defining the len(Dataset) function
  def __len__(self): 
    return len(self.dataframe)

  #defining the _getitem_ function which creates samples, such that when Dataset[i] is called; the i´th sample is returned
  def __getitem__(self, key): 

    if isinstance(key, slice): # if given a slicing
      start, stop, step = key.indices(len(self))
      sliced = deepcopy(self)
      sliced_data = self.dataframe.iloc[start:stop:step]
      sliced.dataframe = sliced_data
      return  sliced

    else: #If given a single index 
      if torch.is_tensor(key):
        key = key.tolist()
      
      #For one sample Dataset[i]:
      dataframe = self.dataframe

      label = dataframe.iloc[key, 2] #retrieveing the label
      dataset = dataframe.iloc[key,0] #retrieveing the dataset-name
      T = dataframe.iloc[key,1] #retrieveing the timeseries-length
      time_series_with_nan = dataframe.iloc[key,3:].to_numpy() #retrieveing the timeseries (containing nan-values at the end)
      time_series = time_series_with_nan[:T] #Removing nan_values at the end

      sample = {'time_series': time_series, 'label': label, 'dataset': dataset, "T": T} #a sample is one timeseries with it's corresponding label (and som xtra information)

      if self.transform: #transform sample
        sample = self.transform(sample)

      return sample
  
  def info(self):#Function to print information about the dataset
    print("Datasets included: ", self.Datasets)
    print("Number of classes : ", self.classes)
    print("Size of dataset: ", len(self))
  
  def shuffle(self):
    self.dataframe = self.dataframe.sample(frac = 1)
  

### Transformation class; segment timeseries into two augmentations
class TwoSegments(object):
  def __init__(self, horizon=0.3, window_gap=1, random_startpos = False, random_horizon=False, random_window_gap=False):
    #horizon: horizon*T = window_length; the length of the two augmentations
    #window_gap: the gap bewteen the two augmentations
    #random_startpos: True means that the first augmentation starts at a random position
    #random_horizon: True means that a random horizon is chosen
    #random_window_gap: True means that a random window_gap is chosen

    self.horizon = horizon
    self.window_gap = window_gap
    self.random_startpos = random_startpos
    self.random_horizon = random_horizon
    self.random_window_gap = random_window_gap
        
  def __call__(self, sample):

    dataset = sample['dataset']
    time_series = sample['time_series']
    T = sample['T']
    label = sample['label']

    #horizon
    if (self.random_horizon==True):
      possible_horizons = [0.15, 0.2, 0.25, 0.3, 0.35, 0.4]
      horizon = random.choice(possible_horizons) #draw a random horizon
    else:
      horizon = self.horizon

    #window gap
    if (self.random_window_gap==True):
      possible_window_gaps =[0,1,2,3,4,5,6,7,8,9,10]
      window_gap = random.choice(possible_window_gaps) #draw random window_gap
    else:
      window_gap = self.window_gap

    #window_length = int(horizon*T) #length of each augmentation
    window_length = 38

    #finding start-position of the first augmentation
    if (self.random_startpos == True): #if random start_position
      max_possible_startposition = T-(2*window_length+window_gap) #the maximal start-position of the first augmentation
      possible_startpossisions = [i for i in range(max_possible_startposition_aug1)] #The possible start positions of the first augmentation
      start_pos = random.choice(possible_startpossisions) #draw a random startposition
    else:
      start_pos = 0 

    #make the two augmentations of the timeseries
    augmentation_1 = time_series[start_pos : (start_pos+window_length)]
    augmentation_2 = time_series[(start_pos+window_length+window_gap) : (start_pos+window_length+window_gap)+window_length]

    #create a new sample with the two augmentations
    #new_sample = {'time_series': time_series, 'aug1': augmentation_1, 'aug2': augmentation_2, 'label': label, 'dataset': dataset, "T": T}
    new_sample = {'aug1': augmentation_1, 'aug2': augmentation_2, 'label': label, 'dataset': dataset, "T": T}

    return new_sample

### Transformation class; convert into Tensor-data for PyTorch-use
class ToTensor(object):
  def __call__(self, sample):
    dataset = sample['dataset']
    T = sample['T']
    label = sample['label']

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    #if (len(sample)== 6):
    if (len(sample)== 5):
      #time_series = sample['time_series'].astype(float)
      #time_series = time_series[np.newaxis,:] #tensor

      aug1 = sample['aug1'].astype(float)
      aug1 = aug1[np.newaxis,:] #tensor

      aug2 = sample['aug2'].astype(float)
      aug2 = aug2[np.newaxis,:] # tensor
      
      #label = label.astype(float)

      #time_series_tensor = torch.tensor(time_series, dtype=torch.float32, device=device)
      aug1_tensor = torch.tensor(aug1, dtype=torch.float32, device=device)
      aug2_tensor = torch.tensor(aug2, dtype=torch.float32, device=device)
      label_tensor = torch.tensor(label, dtype=torch.long, device=device)

      #torch_sample = {'time_series': time_series_tensor, 'aug1': aug1_tensor, 'aug2': aug2_tensor, 'label': label_tensor, 'dataset': dataset, "T": T}
      torch_sample = {'aug1': aug1_tensor, 'aug2': aug2_tensor, 'label': label_tensor, 'dataset': dataset, "T": T}


    else:
      time_series = sample['time_series'].astype(float)
      time_series = time_series[np.newaxis,:] #tensor

      label = label.astype(float)

      time_series_tensor = torch.tensor(time_series, dtype=torch.float32, device=device)
      label_tensor = torch.tensor(label, dtype=torch.long, device=device)

      torch_sample = {'time_series': time_series_tensor, 'label': label_tensor, 'dataset': dataset, "T": T}

    return torch_sample

#Function to split into train and test dataset given a Timeseries_Dataset object
def data_split(dat, train_size=0.7):
  N = len(dat)
  n = int(train_size*N)

  dat.shuffle()
  train = dat[:n]
  test = dat[n:]

  return train, test

#function to save results in Excel
def to_Excel(Results, columns, output_name):
  dataframe = pd.DataFrame(Results, columns = columns)
  # create excel writer object
  writer = pd.ExcelWriter(output_name)
  # write dataframe to excel
  dataframe.to_excel(writer)
  # save the excel
  writer.save()
  files.download(output_name)
  print('DataFrame is written successfully to Excel File.') 

###################



In [26]:
##### MODELS #######

### FCN modellen fra prosjektoppgave
class FCN(nn.Module):
  def __init__(self, class_train=False, classes = 0):
    super(FCN, self).__init__()
    self.class_train= class_train
    self.classes = classes

    self.conv1 = nn.Conv1d(1, 128, 9, padding=(9 // 2))
    #self.bnorm1 = nn.BatchNorm1d(128) 
    self.bnorm1 = nn.GroupNorm(128,128)

    self.conv2 = nn.Conv1d(128, 256, 5, padding=(5 // 2))
    #self.bnorm2 = nn.BatchNorm1d(256)
    self.bnorm2 = nn.GroupNorm(256,256)

    self.conv3 = nn.Conv1d(256, 128, 3, padding=(3 // 2))
    #self.bnorm3 = nn.BatchNorm1d(128)
    self.bnorm3 = nn.GroupNorm(128,128)

    self.output_dim = 128

    if (class_train==True):
      self.classification_head = nn.Linear(128, classes)

  def forward(self, x):
    b1_class = F.relu(self.bnorm1(self.conv1(x)))
    b2_class = F.relu(self.bnorm2(self.conv2(b1_class)))
    b3_class = F.relu(self.bnorm3(self.conv3(b2_class)))

    features_class = torch.mean(b3_class, 2) 

    if (self.class_train==True):
      out_class = self.classification_head(features_class)
      return out_class
    else:
      return features_class


In [27]:
### SIMPLE SIAMESE REPRESENTATION LEANRING MODELLEN ###

## DISTANCE FUNCTION ##
#The distance to minimize  (Negative Cosine similarity)
def D(p, z):
  z = z.detach() #stop gradient
  p = p

  neg_cosine_sim = - F.cosine_similarity(p, z, dim=1) #Negative cosine similarities (size: [bs])

  return neg_cosine_sim.mean()  #return the mean of the negative cosine similarities


## PROJECTION MLP ## ( in f(x))
#has 3 FC layers with BN, the output FC has no ReLU, hidden FC is 2048-d
class projection_MLP(nn.Module):
  def __init__(self, in_dim, hidden_dim=2048, out_dim=2048): 
    super().__init__()
    #Layer 1
    self.FC1 = nn.Sequential(
      nn.Linear(in_dim, hidden_dim),
      nn.BatchNorm1d(hidden_dim),
      nn.ReLU(inplace=True)  )
    #Layer 2
    self.FC2 = nn.Sequential(
      nn.Linear(hidden_dim, hidden_dim),
      nn.BatchNorm1d(hidden_dim),
      nn.ReLU(inplace=True)  )
    #Layer 3 (output)
    self.FC3 = nn.Sequential(
      nn.Linear(hidden_dim, out_dim),
      nn.BatchNorm1d(hidden_dim) )
  
  def forward(self, x):
    x1 = self.FC1(x)
    x2 = self.FC2(x1)
    x_out = self.FC3(x2)
    return x_out


## PREDICTION MLP ## h(z)
#has 2 FC layers, only BN and ReLU on hidden layer (explained why in SimSiam paper). 
# input: z = f(x) dim = 2048
# output: p = h(z) dim = 2048
class prediction_MLP(nn.Module):
  def __init__(self, in_dim=2048, hidden_dim=512, out_dim=2048): # bottleneck structure
    super().__init__()
    #Layer 1
    self.FC1 = nn.Sequential(
      nn.Linear(in_dim, hidden_dim),
      nn.BatchNorm1d(hidden_dim),
      nn.ReLU(inplace=True) )
    #Layer 2 (output)
    self.FC2 = nn.Linear(hidden_dim, out_dim)
  
  def forward(self, x):
    x1 = self.FC1(x)
    x_out = self.FC2(x1)
    return x_out


## SIMSIAM MODEL ##
class SimSiam(nn.Module):
  def __init__(self, backbone=FCN()): #take backbone as input
    super().__init__()
    self.name = SimSiam
    self.backbone = backbone
    self.projector = projection_MLP(backbone.output_dim) 
    
    #Encoder; z = f(x)
    self.encoder = nn.Sequential( 
      self.backbone,
      self.projector )

    #Predictor; p = h(z)
    self.predictor = prediction_MLP()
  
  def forward(self, x1, x2):
    #x1, x2: augmentations of x
    f = self.encoder
    h = self.predictor

    z1, z2 = f(x1), f(x2)
    p1, p2 = h(z1), h(z2)


    #Symmetric loss
    Loss = D(p1, z2)/2 + D(p2, z1)/2

    return Loss

###################


In [39]:
##### TRAINING A SIMSIAM MODEL #####

### Function to train model
def train(model, optimizer, train_data, train_epochs, old_epoch=0, train_bs=512, lr=0.001, SaveName="SimSiam"):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  train_loader = DataLoader(train_data, batch_size = train_bs, shuffle=True)
  train_losses =[]

  for e in range(old_epoch+1, old_epoch+train_epochs):
    losses =[]
    model.train()
    for batch in enumerate(train_loader):
      x1_batch = batch[1]['aug1']
      x2_batch = batch[1]['aug2']

      model.zero_grad()
      loss = model.forward(x1_batch, x2_batch).mean()
      loss.backward()
      optimizer.step()

      losses.append(loss.item())
  
    train_loss = np.mean(losses)
    print("e: ",e," Avg Loss: ", train_loss)
    train_losses.append(train_loss)

    if (e in [ 5, 10, 15, 20, 30, 40, 50, 60, 70, 90]):
      save_checkpoint(SaveName, model, e, optimizer, train_loss, lr, train_bs)

  save_checkpoint(SaveName, model, e, optimizer, train_loss, lr, train_bs)

### Function to save a checkpoint in Google Drive
def save_checkpoint(name, model, epoch, optimizer, train_loss, lr, train_bs):
  drive.mount('/content/drive')
  PATH = f"/content/drive/MyDrive/checkpoints/{name}_{datetime.now().strftime('%d%m')}_ep:{epoch}.pth"
  checkpoint = {'epoch': epoch, 
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'train_loss': train_loss, 
              'lr': lr,
              'train_bs' : train_bs}
  torch.save(checkpoint, PATH) 
  
### Function to load a previously saved checkpoint from Google Drive
def load_checkpoint(name,date, e):
  drive.mount('/content/drive')
  PATH = f"/content/drive/MyDrive/checkpoints/{name}_{date}_ep:{e}.pth"
  checkpoint = torch.load(PATH)

  epoch = checkpoint['epoch']
  train_loss = checkpoint['train_loss']
  #lr = checkpoint['lr']
  #batch_size = checkpoint['batch_size']

  #print("checkpoint:", date," Epoch: ", epoch, " Loss: ", train_loss, "lr: ", lr, "train_bs: ", train_bs)
  print("checkpoint:", date," Epoch: ", epoch, " Loss: ", train_loss)
  return checkpoint

## Function to get a previously trained model from a checkpoint
def load_trained_model(checkpoint):
  epoch = checkpoint['epoch']
  train_loss = checkpoint['train_loss']
  #lr = checkpoint['lr']
  #train_bs = checkpoint['train_bs']

  train_bs = 512
  base_lr = 0.05
  lr = (base_lr*train_bs)/256
  model = SimSiam().to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  model.eval()

  return model.to(device), optimizer, epoch


###################

In [46]:
##### LINEAR EVALUATION: training classifier on top of frozen model and evaluating #####

## Function to calculate classification_accuracy given the test_set
def evaluate_classifier(test_loader, backbone_model, classifier):
  classifier.eval()
  accuracies = []
  for batch in enumerate(test_loader):
    time_series_batch = batch[1]['time_series']
    label_batch = batch[1]['label']

    with torch.no_grad():
      feature = backbone_model(time_series_batch)
      y_hat = classifier(feature)
      pred = torch.max(y_hat,1)[1]

      #print("pred: ", pred)
      #print("true: ", label_batch)
      accuracy = accuracy_score(label_batch, pred) 
      accuracies.append(accuracy)
  
  Accuracy = np.mean(accuracies)
  return Accuracy

## function to calculate classification_accuracy of FCN model (to compare, just for now)
def evaluate_FCN(test_loader, model):
  model.eval()
  accuracies = []
  for batch in enumerate(test_loader):
    time_series_batch = batch[1]['time_series']
    label_batch = batch[1]['label']

    with torch.no_grad():
      y_hat = model(time_series_batch)
      pred = torch.max(y_hat,1)[1]

      #print("pred: ", pred)
      #print("true: ", label_batch)
      accuracy = accuracy_score(label_batch, pred) 
      accuracies.append(accuracy)
  
  Accuracy = np.mean(accuracies)
  return Accuracy


## function to save checkpoint of linear classifier
def save_checkpoint_classifier(SaveName, model, epoch, optimizer, loss_list, accuracy_list, lr, train_bs):
  drive.mount('/content/drive')
  PATH = f"/content/drive/MyDrive/checkpoints/classifier_{SaveName}_{datetime.now().strftime('%d%m')}_ep:{epoch}.pth"
  checkpoint = {'epoch': epoch, 
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss_list': loss_list, 
              'accuracy_list': accuracy_list,
              'lr': lr,
              'train_bs' : train_bs}
  torch.save(checkpoint, PATH)

## function to load checkpoint of linear classifier
def load_checkpoint_classifier(name,date, e):
  drive.mount('/content/drive')
  PATH = f"/content/drive/MyDrive/checkpoints/classifier_{name}_{date}_ep:{e}.pth"
  checkpoint = torch.load(PATH)

  epoch = checkpoint['epoch']
  train_loss = checkpoint['loss_list'][-1]
  max_accuracy = max(checkpoint['accuracy_list'])

  print("checkpoint:", date," Epoch: ", epoch, " Max_accuracy: ",accuracy," Loss: ", train_loss)
  return checkpoint


## Function to do LINEAR EVALUATION of a model trained with SimSiam
def Linear_evaluation(trained_model, classification_dataset, SaveName=None, epochs = 10, train_bs = 512, lr=None):
  #trained model: a model prevoously trained with SimSiam
  #classification_dataset: The dataset to train and evaluate the classification 

  dataset = classification_dataset.Datasets[0]
  train_class, test_class = data_split(classification_dataset, train_size=0.5) #Split into a 

  if(lr==None):
    base_lr = 0.05
    lr = (base_lr*train_bs)/256

  train_loader = DataLoader(train_class, batch_size = train_bs, shuffle=True)
  test_loader = DataLoader(test_class, batch_size = train_bs, shuffle=True)

  classes = test_class.classes

  #Get the frozen backbone of the trained model
  Backbone_model = trained_model.backbone
  Backbone_model = Backbone_model.to(device)
  Backbone_model = nn.DataParallel(Backbone_model)

  #Linear classifier to train in top of frozen model
  Linear = nn.Linear(128, classes).to(device)
  optimizer_linear = torch.optim.Adam(Linear.parameters(), lr=lr)
  accuracies_linear = []
  losses_linear = []

  #FCN model to compare accuracy (just for now..)
  FCN_model = FCN(class_train=True, classes = classes).to(device)
  optimizer_FCN = torch.optim.Adam(FCN_model.parameters(), lr=lr)
  accuracies_FCN =[]
  losses_FCN =[]

  for e in range(epochs):
    Backbone_model.eval()
    Linear.train()
    loss_list_linear = []
    
    FCN_model.train()
    loss_list_FCN = []

    for batch in enumerate(train_loader):
      time_series_batch = batch[1]['time_series']
      label_batch = batch[1]['label']

      #Train linear classifier on top of backbone model
      Linear.zero_grad()
      with torch.no_grad():
        feature = Backbone_model(time_series_batch.to(device))
      pred = Linear(feature.to(device))
      loss_linear = F.cross_entropy(pred, label_batch)
      loss_linear.backward()
      optimizer_linear.step()
      loss_list_linear.append(loss_linear.item())
      
      #train FCN model
      y_hat = FCN_model(time_series_batch.to(device))
      loss_FCN = F.cross_entropy(y_hat, label_batch)
      optimizer_FCN.zero_grad()
      loss_FCN.backward()
      optimizer_FCN.step()
      loss_list_FCN.append(loss_FCN.item())

    Loss_linear = np.mean(loss_list_linear)
    losses_linear.append(Loss_linear)
    accuracy_linear = evaluate_classifier(test_loader, Backbone_model, Linear)
    accuracies_linear.append(accuracy_linear)

    Loss_FCN = np.mean(loss_list_FCN)
    losses_FCN.append(Loss_FCN)
    accuracy_FCN = evaluate_FCN(test_loader, FCN_model)
    accuracies_FCN.append(accuracy_FCN)

    #print("e:  ", e, "SimSiam : accuracy; ",accuracy_linear, " loss; ",Loss_linear )
    print("e:  ", e, " dataset: ", dataset, " : accuracy; ",accuracy_linear, " loss; ",Loss_linear, "   FCN: accuracy; ", accuracy_FCN, " loss; ", Loss_FCN)

  results = [dataset, "linear eval", e , train_bs, lr, accuracies_linear[-1], max(accuracies_linear), losses_linear[-1], max(accuracies_FCN)]

  if (SaveName !=None):
    save_checkpoint_classifier(SaveName, Linear, e, optimizer_linear, losses_linear, accuracies_linear, lr, train_bs) #Save checkpoint
  
  return results


In [47]:
## Loading dataset

#Dataset_small = Datasets = ["ChlorineConcentration"]
#Small_dataset = Timeseries_Dataset(Dataset_small, transform = transforms.Compose( [TwoSegments(), ToTensor()] ))
#train_small, test_small = data_split(Small_dataset)

#Dataset_big = ["Big_dataset"]
#Big_dataset = Timeseries_Dataset(Dataset_big, Drive=True, transform = transforms.Compose( [TwoSegments(), ToTensor()] ))
#train_big, test_big = data_split(Big_dataset)

#date=1904

Big_dataset = ["ChlorineConcentration", "Two_Patterns", "yoga", "uWaveGestureLibrary_X", "NonInvasiveFatalECG_Thorax1", "ECG5000", "FordA", "FordB"]
Save_BigDataset = "BigDataset"
SaveName_BigDataset = "BigDataset"
#BigDatasets_train = Timeseries_Dataset(Big_dataset, train=True, Save= None, transform = transforms.Compose( [TwoSegments(), ToTensor()] ))
epoch_big = 30


Two_datasets = ["ChlorineConcentration", "Two_Patterns"]
Save_TwoDatasets = "TwoDatasets_train"
SaveName_TwoDatasets = "TwoDatasets"
#TwoDatasets_train = Timeseries_Dataset(Two_datasets, train=True, Save= Save, transform = transforms.Compose( [TwoSegments(), ToTensor()] ))
epoch_two = 50



date =2004
epoch_big2 = 99
SaveName_BigDataset2 = "BigDataset2"




In [ ]:
### Training a SimSiam model
SaveName_BigDataset2 = "BigDataset2"
epochs = 100
train_bs = 512

base_lr = 0.05
lr = (base_lr*train_bs)/256

model = SimSiam().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

train(model, optimizer, BigDatasets_train, epochs, old_epoch=0, train_bs=train_bs, lr=lr, SaveName=SaveName_BigDataset2)


In [ ]:
### Loading previously trained models

#SaveName = "SimSiam"
#checkpoint = load_checkpoint(SaveName, 1304, 99) #Loading the checkpoint that has trained 99 epochs with the train_small training set. 
#model_99, optimizer_99, epoch_99 = load_trained_model(checkpoint)

#continue training previously trained model
#train(model_99, optimizer_99, train_small, 2, old_epoch=epoch_99, train_bs=512, SaveName="SimSiam")


#checkpoint_two = load_checkpoint(SaveName_TwoDatasets, date, epoch_two)
#model_two, optimizer_two, epoch_two = load_trained_model(checkpoint_two)

#checkpoint_big = load_checkpoint(SaveName_BigDataset, date, epoch_big)
#model_big, optimizer_big, epoch_big = load_trained_model(checkpoint_big)


checkpoint_big2 = load_checkpoint(SaveName_BigDataset2, date, epoch_big2)
model_big2, optimizer_big2, epoch_big2 = load_trained_model(checkpoint_big2)



In [ ]:

Big_dataset = ["ChlorineConcentration", "Two_Patterns", "yoga", "uWaveGestureLibrary_X", "NonInvasiveFatalECG_Thorax1", "ECG5000", "FordA", "FordB"]


Results = []
for dataset in Big_dataset:
  classification_dataset = Timeseries_Dataset([dataset], train=False, Save=None, transform = transforms.Compose( [ ToTensor()] ))
  train_class, test_class = data_split(classification_dataset, train_size=0.5)
  SaveName = dataset

  results = Linear_evaluation(model_big2, classification_dataset, SaveName=None, epochs = 20, train_bs = 40, lr=None)
  results.append(SaveName_BigDataset)
  results.append(date)
  
  Results.append(results)

column_names = ["Dataset", "Type", "epochs", "batch_size", "lr", "Accuracy", "Max Accuracy", "Loss", "FCN_accuracy", "trained model used", "date trained model"]

to_Excel(Results, column_names, "Checking_LinearEval.xlsx")


